<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Data_Mining_Module1_AMZ_CLV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Customer Lifetime Value model

In [5]:
#package used for customer lifetime value modeling
!pip install lifetimes

In [6]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling


In [8]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)

# Import the lifetimes package for customer lifetime value analysis.
import lifetimes
# Import pandas for data manipulation and analysis.
import pandas as pd
# Import numpy for numerical computing.
import numpy as np
# Import datetime for handling dates and times.
import datetime as dt
# Import matplotlib.pyplot for creating static, animated, and interactive visualizations.
import matplotlib.pyplot as plt
# Import seaborn for making statistical graphics.
import seaborn as sns
# Import BetaGeoFitter from lifetimes for fitting the BG/NBD model.
from lifetimes import BetaGeoFitter
# Import GammaGammaFitter from lifetimes for fitting the Gamma-Gamma model.
from lifetimes import GammaGammaFitter
# Import plot_frequency_recency_matrix for visualizing the relationship between frequency and recency.
from lifetimes.plotting import plot_frequency_recency_matrix

# Set pandas display options to improve the readability of output.
# Set the maximum number of rows to display to 500.
pd.set_option('display.max_rows', 500)
# Set the maximum number of columns to display to 500.
pd.set_option('display.max_columns', 500)
# Set the width of the display in characters to 1000.
pd.set_option('display.width', 1000)

import requests, zipfile, io

# GitHub URL of the raw .zip file
file_url = 'https://github.com/Requenamar3/Data-Mining/raw/main/Amazon%20Sale%20Report.zip'

# Send a GET request to the file URL
r = requests.get(file_url, stream=True)

# Check if the request was successful
if r.status_code == 200:
    # Open the content of the response as a byte stream and pass it to ZipFile
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        # Extract all the contents into the current working directory
        z.extractall()
    print("File downloaded and extracted successfully.")
else:
    # If the response status code is not 200, print the status code
    print(f"Error downloading the file, status code: {r.status_code}")


File downloaded and extracted successfully.


In [10]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

In [9]:
# Print out the names of the files in the zip to determine what you have extracted
print(z.namelist())


['Amazon Sale Report.csv']


In [13]:
# Read the specified columns from the CSV
AMZ = pd.read_csv('Amazon Sale Report.csv')

AMZ.T


<ipython-input-13-6f2f1a69a11b>:9: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  AMZ = pd.read_csv('Amazon Sale Report.csv')
<ipython-input-13-6f2f1a69a11b>:20: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  AMZ = pd.read_csv('Amazon Sale Report.csv')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,128725,128726,128727,128728,128729,128730,128731,128732,128733,128734,128735,128736,128737,128738,128739,128740,128741,128742,128743,128744,128745,128746,128747,128748,128749,128750,128751,128752,128753,128754,128755,128756,128757,128758,128759,128760,128761,128762,128763,128764,128765,128766,128767,128768,128769,128770,128771,128772,128773,128774,128775,128776,128777,128778,128779,128780,128781,128782,128783,128784,128785,128786,128787,128788,128789,128790,128791,128792,128793,128794,128795,128796,128797,128798,128799,128800,128801,128802,128803,128804,128805,128806,128807,128808,128809,128810,128811,128812,128813,128814,128815,128816,128817,128818,128819,128820,128821,128822,128823,128824,128825,128826,128827,128828,128829,128830,128831,128832,128833,128834,128835,128836,128837,128838,128839,128840,128841,128842,128843,128844,128845,128846,128847,128848,128849,128850,128851,128852,128853,128854,128855,128856,128857,128858,128859,128860,128861,128862,128863,128864,128865,128866,128867,128868,128869,128870,128871,128872,128873,128874,128875,128876,128877,128878,128879,128880,128881,128882,128883,128884,128885,128886,128887,128888,128889,128890,128891,128892,128893,128894,128895,128896,128897,128898,128899,128900,128901,128902,128903,128904,128905,128906,128907,128908,128909,128910,128911,128912,128913,128914,128915,128916,128917,128918,128919,128920,128921,128922,128923,128924,128925,128926,128927,128928,128929,128930,128931,128932,128933,128934,128935,128936,128937,128938,128939,128940,128941,128942,128943,128944,128945,128946,128947,128948,128949,128950,128951,128952,128953,128954,128955,128956,128957,128958,128959,128960,128961,128962,128963,128964,128965,128966,128967,128968,128969,128970,128971,128972,128973,128974
index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,128725,128726,128727,128728,128729,128730,128731,128732,128733,128734,128735,128736,128737,128738,128739,128740,128741,128742,128743,128744,128745,128746,128747,128748,128749,128750,128751,128752,128753,128754,128755,128756,128757,128758,128759,128760,128761,128762,128763,128764,128765,128766,128767,128768,128769,128770,128771,128772,128773,128774,128775,128776,128777,128778,128779,128780,128781,128782,128783,128784,128785,128786,128787,128788,128789,

In [14]:
# Define the columns you want to read
columns_to_read = [
    'Order ID', 'Date', 'Status', 'Qty','Amount',
    'SKU'
]

# Read the specified columns from the CSV
AMZ = pd.read_csv("Amazon Sale Report.csv", usecols=columns_to_read)

AMZ.head()


,Order ID,Date,Status,SKU,Qty,Amount
0,405-8078784-5731545,04-30-22,Cancelled,SET389-KR-NP-S,0,647.62
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,JNE3781-KR-XXXL,1,406.00
2,404-0687676-7273146,04-30-22,Shipped,JNE3371-KR-XL,1,329.00
3,403-9615377-8133951,04-30-22,Cancelled,J0341-DR-L,0,753.33
4,407-1069790-7240320,04-30-22,Shipped,JNE3671-TU-XXXL,1,574.00


In [16]:
AMZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Order ID  128975 non-null  object 
 1   Date      128975 non-null  object 
 2   Status    128975 non-null  object 
 3   SKU       128975 non-null  object 
 4   Qty       128975 non-null  int64  
 5   Amount    121180 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 5.9+ MB


In [17]:
AMZ.describe()

,Qty,Amount
count,128975.000000,121180.000000
mean,0.904431,648.561465
std,0.313354,281.211687
min,0.000000,0.000000
25%,1.000000,449.000000
50%,1.000000,605.000000
75%,1.000000,788.000000
max,15.000000,5584.000000


In [18]:
# Create a ProfileReport object
profile = ProfileReport(AMZ, title="AMZ CLV", explorative=True)

In [19]:
# display report
profile


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# Remove rows from 'EC' where 'total_sales' is equal to 0.00
AMZ= AMZ[AMZ['Amount'] != 0.00]


In [22]:
# Remove rows from 'EC' where 'total_sales' is equal to 0.00
AMZ= AMZ[AMZ['Qty'] != 0.00]

In [23]:
AMZ.head()

,Order ID,Date,Status,SKU,Qty,Amount
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,JNE3781-KR-XXXL,1,406.0
2,404-0687676-7273146,04-30-22,Shipped,JNE3371-KR-XL,1,329.0
4,407-1069790-7240320,04-30-22,Shipped,JNE3671-TU-XXXL,1,574.0
5,404-1490984-4578765,04-30-22,Shipped,SET264-KR-NP-XL,1,824.0
6,408-5748499-6859555,04-30-22,Shipped,J0095-SET-L,1,653.0


In [ ]:
EC.isnull().sum()

month                      0
financial_status           0
order_name                 0
customer_email           102
customer_type              0
customer_id                0
variant_sku               17
product_title              0
shipping_postal_code       0
shipping_region            0
customer_cohort_month      0
total_sales                0
ordered_item_quantity      0
dtype: int64

In [ ]:
EC.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [ ]:
EC.isnull().sum()

month                    0
financial_status         0
order_name               0
customer_email           0
customer_type            0
customer_id              0
variant_sku              0
product_title            0
shipping_postal_code     0
shipping_region          0
customer_cohort_month    0
total_sales              0
ordered_item_quantity    0
dtype: int64

In [ ]:
EC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36812 entries, 1 to 131973
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   month                  36812 non-null  object 
 1   financial_status       36812 non-null  object 
 2   order_name             36812 non-null  object 
 3   customer_email         36812 non-null  object 
 4   customer_type          36812 non-null  object 
 5   customer_id            36812 non-null  int64  
 6   variant_sku            36812 non-null  object 
 7   product_title          36812 non-null  object 
 8   shipping_postal_code   36812 non-null  object 
 9   shipping_region        36812 non-null  object 
 10  customer_cohort_month  36812 non-null  object 
 11  total_sales            36812 non-null  float64
 12  ordered_item_quantity  36812 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 3.9+ MB


In [ ]:
EC.describe()

,customer_id,total_sales,ordered_item_quantity
count,3.681200e+04,36812.000000,36812.000000
mean,5.602558e+12,120.834550,1.005487
std,1.691035e+12,152.670425,0.086101
min,1.183560e+12,1.920000,1.000000
25%,5.048806e+12,49.990000,1.000000
50%,6.381719e+12,60.350000,1.000000
75%,7.050673e+12,85.790000,1.000000
max,7.297975e+12,3507.540000,5.000000


In [ ]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [ ]:
capping_outliers(EC,'total_sales')
capping_outliers(EC,'ordered_item_quantity')

In [ ]:
EC.describe()

,customer_id,total_sales,ordered_item_quantity
count,3.681200e+04,36812.000000,36812.0
mean,5.602558e+12,114.139838,1.0
std,1.691035e+12,123.928098,0.0
min,1.183560e+12,42.780000,1.0
25%,5.048806e+12,49.990000,1.0
50%,6.381719e+12,60.350000,1.0
75%,7.050673e+12,85.790000,1.0
max,7.297975e+12,515.060000,1.0


#Recency Frecuency Time

In [ ]:
# Calculate and assign the floor division of 'total_sales' by 'ordered_item_quantity' to 'UnitPrice' column in EC DataFrame.
EC['UnitPrice'] = EC['total_sales'] // EC['ordered_item_quantity']


In [ ]:
EC.columns

Index(['month', 'financial_status', 'order_name', 'customer_email', 'customer_type', 'customer_id', 'variant_sku', 'product_title', 'shipping_postal_code', 'shipping_region', 'customer_cohort_month', 'total_sales', 'ordered_item_quantity', 'UnitPrice'], dtype='object')

In [ ]:
clv = lifetimes.utils.summary_data_from_transaction_data(EC,'customer_id','month','total_sales',observation_period_end='2024-03-05')

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
customer_id,,,,
1183559942259,2.0,62.0,95.0,85.59
1183569084531,1.0,30.0,125.0,42.78
1183569707123,4.0,121.0,125.0,47.07
1183569805427,0.0,0.0,64.0,0.00
1183579504755,0.0,0.0,64.0,0.00


In [ ]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [ ]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months
customer_id,,,,,
1183559942259,2.0,62.0,95.0,85.59,4.317409
1183569707123,4.0,121.0,125.0,47.07,4.847395
1183582322803,3.0,92.0,125.0,53.49,4.433483
1183584551027,4.0,121.0,125.0,43.79,4.847395
1183594184819,4.0,121.0,125.0,43.99,4.847395


#BG/NBD Model

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)  # This will allow you to predict FUTURE purchase by taking into account F, R, T  penalizer is used to avoid overfitting
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

<lifetimes.BetaGeoFitter: fitted with 3988 subjects, a: 0.00, alpha: 309.88, b: 0.00, r: 7.71>

In [ ]:
t = 180 # 6 month/180 day period
# Predict future purchases.
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
# Top 5 by expected purchases.
clv.sort_values(by='expected_purc_6_months',ascending=False).head(5)

,frequency,recency,T,monetary_value,expected_purc_6_months
customer_id,,,,,
5675202707571,4.0,121.0,125.0,52.9900,4.847395
3370717184115,4.0,121.0,125.0,95.9125,4.847395
3360257015923,4.0,121.0,125.0,52.9900,4.847395
6278222839923,4.0,121.0,125.0,52.9900,4.847395
6276989780083,4.0,121.0,125.0,55.3775,4.847395


#Gamma-Gamma Model

In [ ]:
# Calculate and display the correlation between 'frequency' and 'monetary_value' columns in the 'clv' DataFrame.
clv[['frequency', 'monetary_value']].corr()


,frequency,monetary_value
frequency,1.000000,-0.119109
monetary_value,-0.119109,1.000000


In [ ]:
# Initialize GammaGammaFitter with a penalizer to prevent overfitting and fit on frequency and monetary value.
ggf = GammaGammaFitter(penalizer_coef=0.01)  # Create GammaGammaFitter model instance with a specified penalizer.
ggf.fit(clv["frequency"], clv["monetary_value"])  # Fit model to customer data for predicting monetary value.


<lifetimes.GammaGammaFitter: fitted with 3988 subjects, p: 4.38, q: 0.67, v: 4.27>

In [ ]:
# Calculate 6-month Customer Lifetime Value (CLV) using the GammaGamma and BetaGeo models, then identify top customers.
# The 'customer_lifetime_value' method from GammaGammaFitter ('ggf') combines both frequency/recency (from 'bgf') and
# monetary value to estimate the CLV for the next 6 months ('time=6'), assuming daily ('freq='D'') transactions.
# A discount rate of 0.01 is applied to account for the time value of money.
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
customer_id,,,,,,
6543258091635,4.0,121.0,125.0,1302.3025,4.847395,6219.433142
5452489425011,2.0,62.0,95.0,677.9300,4.317409,2946.752548
6424284659827,2.0,62.0,95.0,639.9050,4.317409,2781.987954
5394003755123,2.0,62.0,95.0,622.3000,4.317409,2705.704438
6403314614387,2.0,92.0,125.0,557.0700,4.019571,2255.903477


In [ ]:
# Segment customers into 4 categories based on their CLV using quartiles.
clv['Segment'] = pd.qcut(clv['six_Months_CLV'], 4, labels=['Hibernating', 'Need Attention', 'Loyal Customers', 'Champions'])  # Assign segments based on CLV quartiles.


In [ ]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV,Segment
customer_id,,,,,,,
1183559942259,2.0,62.0,95.0,85.59,4.317409,380.107976,Champions
1183569707123,4.0,121.0,125.0,47.07,4.847395,229.697836,Need Attention
1183582322803,3.0,92.0,125.0,53.49,4.433483,241.194243,Need Attention
1183584551027,4.0,121.0,125.0,43.79,4.847395,214.046288,Hibernating
1183594184819,4.0,121.0,125.0,43.99,4.847395,215.000651,Hibernating


In [ ]:
# Calculate the mean of each column within the 'clv' DataFrame grouped by 'Segment'.
clv.groupby('Segment').mean()


,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
Segment,,,,,,
Hibernating,3.082341,95.632937,124.641865,46.714883,4.471261,212.928824
Need Attention,3.131846,97.695740,124.390467,53.918528,4.494251,245.775800
Loyal Customers,2.778335,87.634905,115.423270,66.139875,4.437674,298.362375
Champions,2.990973,95.603811,119.536610,138.152765,4.485550,618.387546


#Check out top customers

In [ ]:
# Filter out the unwanted product titles before the groupby operation
EC_filtered = EC[~EC['variant_sku'].isin(['SQ3100795', 'SQ5173256'])]

# Group by 'customer_id', 'customer_cohort_month', and 'variant_sku' then count the number of records
customer_record_counts = EC_filtered.groupby(['customer_id', 'customer_cohort_month', 'variant_sku']).size().reset_index(name='record_count')

# Get the top 15 customer IDs by record count in descending order
top_customers = customer_record_counts.nlargest(15, 'record_count')

# drop duplicates
top_customers = top_customers.drop_duplicates(subset='customer_id')

# Merge top_customers with the filtered dataframe to include customer emails and product titles.
# left join to ensure all top customers are included in the final DataFrame even if they have no matching records in EC_filtered
top_customers_info = top_customers.merge(
    EC_filtered[['customer_id', 'customer_cohort_month', 'variant_sku', 'customer_email', 'product_title']],
    on=['customer_id', 'customer_cohort_month', 'variant_sku'],
    how='left'
)

# Select columns we're interested in and drop duplicates
top_customers_info = top_customers_info[['customer_id', 'customer_email', 'product_title', 'variant_sku', 'customer_cohort_month', 'record_count']].drop_duplicates(subset='customer_id')

print(top_customers_info)



       customer_id             customer_email                      product_title  variant_sku customer_cohort_month  record_count
0    6543258091635               na@caroo.com               Singapore Money Tree      BB1352P               2023-03            40
40   6244825202803  Myrna.Flores@azervets.com                    Pet-Safe Blooms  BB-PET-SAFE               2022-09            18
58   2292822999155      lla@adlawcolorado.com                   Bloomsy Original    SQ9178530               2019-09            17
75   3561498411123   missiecrosby99@gmail.com                     Bloomsy Deluxe    SQ1726938               2020-12            16
91   6567567130739             rbsmds@aol.com                   Bloomsy Original    SQ9178530               2023-04            15
106  3548725215347       gwenhargrove@aol.com              The NYBG Subscription     BBNYBGLA               2020-12            14
120  2779612053619         rlcolligan@mac.com            Vacation in the Tropics     BB981

In [ ]:
EC.describe()

,customer_id,total_sales,ordered_item_quantity
count,3.681200e+04,36812.000000,36812.000000
mean,5.602558e+12,120.834550,1.005487
std,1.691035e+12,152.670425,0.086101
min,1.183560e+12,1.920000,1.000000
25%,5.048806e+12,49.990000,1.000000
50%,6.381719e+12,60.350000,1.000000
75%,7.050673e+12,85.790000,1.000000
max,7.297975e+12,3507.540000,5.000000


In [ ]:
EC[]

#Add table from Ship Station in order to look up fo the missing information

In [ ]:
# Load the dataset from the provided URL
ss_df = pd.read_csv('https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/SS.csv')

# Display the first few rows of the dataframe to understand its structure
ss_df.head()

,Date - Shipped Date,Market - Store Name,Market - Markeplace Name,Date - Order Date,Custom - Field 3,Customer Email,Ship To - City,Custom - Field 1,Custom - Field 2,Item - SKU,Ship To - Postal Code,Order - Number,Item - Name,Ship To - State,Item - Qty,Carrier - Service Selected
0,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,8/3/2023 2:49:14 PM,11-01-2023,JDJVJ@AOL.COM,FT LAUDERDALE,4.MIAMI,ORIGINAL_PLEGABLE,SQ9178530,33316-3228,776024,ORIGINAL,FL,1,FedEx Home Delivery®
1,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,8/22/2023 9:48:07 AM,11-01-2023,mary.c.parker@ttuhsc.edu,LUBBOCK,2.TEXAS,DELUXE,BB-SG-MIXED,79416-1340,783802,DELUXE,TX,1,FedEx Home Delivery®
2,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/4/2023 12:56:49 PM,11-01-2023,ajbroc@me.com,CHAPPAQUA,3.NEW JERSEY,DELUXE,BB-SG-MIXED,10514-1302,789981,DELUXE,NY,1,FedEx Home Delivery®
3,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/5/2023 11:59:05 AM,11-01-2023,bndudley52@gmail.com,SCOTTSDALE,5.CALIFORNIA,PREMIUM,SQ8862610,85255-7165,790118,PREMIUM,AZ,1,OnTrac Ground Service
4,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/5/2023 11:59:05 AM,11-01-2023,bndudley52@gmail.com,SCOTTSDALE,5.CALIFORNIA,PREMIUM,NaN,85255-7165,790118,cs15,AZ,1,OnTrac Ground Service


In [ ]:
ss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Date - Shipped Date         469 non-null    object
 1   Market - Store Name         469 non-null    object
 2   Market - Markeplace Name    469 non-null    object
 3   Date - Order Date           469 non-null    object
 4   Custom - Field 3            469 non-null    object
 5   Customer Email              446 non-null    object
 6   Ship To - City              469 non-null    object
 7   Custom - Field 1            451 non-null    object
 8   Custom - Field 2            377 non-null    object
 9   Item - SKU                  455 non-null    object
 10  Ship To - Postal Code       469 non-null    object
 11  Order - Number              469 non-null    object
 12  Item - Name                 469 non-null    object
 13  Ship To - State             469 non-null    object

In [ ]:
ss_df.columns


Index(['Date - Shipped Date', 'Market - Store Name', 'Market - Markeplace Name', 'Date - Order Date', 'Custom - Field 3', 'Customer Email', 'Ship To - City', 'Custom - Field 1', 'Custom - Field 2', 'Item - SKU', 'Ship To - Postal Code', 'Order - Number', 'Item - Name', 'Ship To - State', 'Item - Qty', 'Carrier - Service Selected'], dtype='object')

In [ ]:
# Merge EC_df with ss_df on the 'order_name' column in EC_df and 'Order Number' in ss_df
# Make sure the column names are exactly as they appear in your dataframes
ECSS = EC.merge(ss_df[['Custom - Field 3', 'Customer Email', 'Item - SKU', 'Order - Number', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected']],
                        left_on='order_name', right_on='Order - Number', how='left')

# # Now merged_df has the original EC_df data along with the additional columns from ss_df
# # You can then drop the 'Order - Number' column if it's no longer needed
# merged_df.drop('Order - Number', axis=1, inplace=True)

# # If you only want to keep rows where data was successfully merged and no NaN values exist in the new columns, you can do this:
# merged_df.dropna(subset=['Custom - Field 3', 'Customer Email', 'Item - SKU', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected'], inplace=True)

# Save the merged dataframe to a new CSV file or use it for further analysis
# merged_df.to_csv('merged_dataframe.csv', index=False)

ECSS.head()  # Display the first few rows of the merged dataframe


,month,financial_status,order_name,customer_email,customer_type,customer_id,variant_sku,product_title,shipping_postal_code,shipping_region,customer_cohort_month,total_sales,ordered_item_quantity,Custom - Field 3,Customer Email,Item - SKU,Order - Number,Item - Name,Ship To - State,Carrier - Service Selected
0,2023-11,paid,826267,amansndrs@gmail.com,Returning,6816167329907,NaN,NaN,71254,Louisiana,2023-08,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11,paid,821120,jsajcat@gmail.com,Returning,6882693939315,SQ1726938,Bloomsy Deluxe,60101,Illinois,2023-09,76.98,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11,paid,825181,amymattey@gmail.com,Returning,3492230627443,NaN,NaN,44035,Ohio,2020-11,83.88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11,partially_refunded,793583,kathryndonnelly@yahoo.com,Returning,6536856797299,NaN,NaN,60555,Illinois,2023-03,-162.24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11,partially_refunded,814967,nancynetherland@gmail.com,Returning,5686113435763,NaN,NaN,98502,Washington,2022-05,-91.44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ECSS.columns

Index(['month', 'financial_status', 'order_name', 'customer_email', 'customer_type', 'customer_id', 'variant_sku', 'product_title', 'shipping_postal_code', 'shipping_region', 'customer_cohort_month', 'total_sales', 'ordered_item_quantity', 'Custom - Field 3', 'Customer Email', 'Item - SKU', 'Order - Number', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected'], dtype='object')

In [ ]:
ECSS.isnull().sum()

month                              0
financial_status                   0
order_name                         0
customer_email                  3049
customer_type                      0
customer_id                        0
variant_sku                    63281
product_title                  63264
shipping_postal_code               1
shipping_region                    1
customer_cohort_month              0
total_sales                        0
ordered_item_quantity              0
Custom - Field 3              131970
Customer Email                131970
Item - SKU                    131970
Order - Number                131970
Item - Name                   131970
Ship To - State               131970
Carrier - Service Selected    131970
dtype: int64

In [ ]:
df.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [ ]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64